In [1]:
# script that assembles all data necessary to draw networks per tissue highlighting the 
# causal gene - TiP gene interactions we found in HI-III and edgotyping results on in

import igraph
import urllib, os, subprocess
import requests
import json
from IPython.display import display,Image
from py2cytoscape import util as cy
from py2cytoscape.data.cyrest_client import CyRestClient
cy = CyRestClient()
from py2cytoscape import cyrest

import CG_TiP_PPI_drawing_classes
import database_utils

connect = database_utils.get_connection()
cursor = connect.cursor()

PORT_NUMBER = 1234
IP = 'localhost'
BASE = 'http://' + IP + ':' + str(PORT_NUMBER) + '/v1/'
HEADERS = {'Content-Type':'application/json'}

outpath = '../data/katjas_data/tissue_spec_disease_edgotyping/'

In [2]:
# function that writes out the nodes and their attributes from the given graph object in cytoscape readable format
def write_node_attributes(graph,outfile):

    target = open(outfile,'w')
    node_attributes = graph.vertex_attributes()
    node_attributes.sort()
    target.write('geneID')
    for attr in node_attributes:
        if attr != 'name':
            target.write('\t' + attr)
    target.write('\n')

    for node in graph.vs:
        target.write(node['name'])
        for attr in node_attributes:
            if attr != 'name':
                if isinstance(node[attr],list):
                    outstring = ''
                    for value in node[attr]:
                        outstring = outstring + str(value) + '|'
                    outstring = outstring[:-1]
                else:
                    outstring = str(node[attr])
                target.write('\t' + outstring)
        target.write('\n')

    target.close()


# function that writes out the edges and their attributes from the given graph object in cytoscape readable format
def write_edge_attributes(graph,outfile):

    target = open(outfile,'w')
    edge_attributes = graph.edge_attributes()
    edge_attributes.sort()
    target.write('geneID_A\tgeneID_B')
    for attr in edge_attributes:
        target.write('\t' + attr)
    target.write('\n')

    for edge in graph.es:
        source_node = graph.vs[edge.source]['name']
        target_node = graph.vs[edge.target]['name']
        if source_node < target_node:
            target.write(str(source_node) + '\t' + str(target_node))
        else:
            target.write(str(target_node) + '\t' + str(source_node))
        for attr in edge_attributes:
            if isinstance(edge[attr],list):
                outstring = ''
                for value in edge[attr]:
                    outstring = outstring + str(value) + '|'
                outstring = outstring[:-1]
            else:
                outstring = str(edge[attr])
            target.write('\t' + outstring)
        target.write('\n')

    target.close()


In [3]:
# load the data
std_batches = ['tipAs1','tipAs2','tipAs6']
assay_versions = [1,2,6]
data = CG_TiP_PPI_drawing_classes.get_hi_iii_CG_TiP_data(std_batches)

Number of diseases loaded: 1105
ensembl_gene_id,`adipose_subcutaneous`,`adipose_visceral_omentum`,`adrenal_gland`,`artery_aorta`,`artery_coronary`,`artery_tibial`,`brain-0`,`brain-1`,`brain-2`,`breast_mammary_tissue`,`colon_sigmoid`,`colon_transverse`,`esophagus_gastroesophageal_junction`,`esophagus_mucosa`,`esophagus_muscularis`,`heart_atrial_appendage`,`heart_left_ventricle`,`kidney_cortex`,`liver`,`lung`,`minor_salivary_gland`,`muscle_skeletal`,`nerve_tibial`,`ovary`,`pancreas`,`pituitary`,`prostate`,`skin`,`small_intestine_terminal_ileum`,`spleen`,`stomach`,`thyroid`,`uterus`,`vagina`,`whole_blood`
Number of genes loaded: 8275
Number of alleles loaded: 50
Number of HI-III interactions loaded: 52569
Number of allele interactions loaded: 564


In [4]:
tissue_graphs = {}
# for every tissue:
for t,tissue in enumerate(data.tissues):
    print(tissue)
    nodes = set()
    edges = set()
    CGs = set()
    # find all causal genes that affect this tissue, that are not a TiP gene in this tissue 
    # and that have at least 1 PPI in HI-III to a TiP gene in this tissue:
    # for every disease:
    for omim_id,disease_obj in data.diseases.items():
        if tissue in disease_obj.tissues:
            # for every CG-tissue pair that matches this tissue
            for CGtis_pair in disease_obj.CG_tissue_pairs:
                gene_id = CGtis_pair[0]
                dis_tissue = CGtis_pair[1]
                if dis_tissue == tissue and gene_id in data.genes:
                    # if this CG is expressed in this tissue and it is not a TiP gene in this tissue
                    if len(data.genes[gene_id].tissue_expr_values) > 0 and \
                       data.genes[gene_id].tissue_expr_values[t] > 5 and \
                       data.genes[gene_id].tissue_TiP_values[t] < 2:
                        # does it have any PPI in HI-III to a TiP gene in this tissue?
                        found = 0
                        for int_gene_id in data.genes[gene_id].PPIs.keys():
                            if len(data.genes[int_gene_id].tissue_expr_values) > 0 and \
                               data.genes[int_gene_id].tissue_expr_values[t] > 5 and \
                               data.genes[int_gene_id].tissue_TiP_values[t] >= 2:
                                found = 1
                                break
                        if found == 1:
                            # save causal gene
                            nodes.add(gene_id)
                            CGs.add(gene_id)
                            # save all interaction partners of that causal gene that are also expressed in this tissue
                            for int_gene_id in data.genes[gene_id].PPIs.keys():
                                if len(data.genes[int_gene_id].tissue_expr_values) > 0 and \
                                   data.genes[int_gene_id].tissue_expr_values[t] > 5:
                                    nodes.add(int_gene_id)
                                    edges.add(tuple(sorted([gene_id,int_gene_id])))

    # save all nodes and edges in graph
    g = igraph.Graph()
    nodes = list(nodes)
    edges = list(edges)
    g.add_vertices(nodes)
    g.add_edges(edges)
    g.vs['orf_id'] = [data.genes[gene_id].orf_ids[0] for gene_id in nodes]
    g.vs['symbol'] = [data.genes[gene_id].symbol for gene_id in nodes]
    g.vs['descr'] = [data.genes[gene_id].descr for gene_id in nodes]
    g.vs['CG'] = [(1 if gene_id in CGs else 0) for gene_id in nodes]
    g.vs['HI_degree'] = [len(data.genes[gene_id].PPIs) for gene_id in nodes]
    g.vs['weight'] = [1.0/len(data.genes[gene_id].PPIs) for gene_id in nodes]
    g.vs['expr_value'] = [data.genes[gene_id].tissue_expr_values[t] for gene_id in nodes]
    g.vs['TiP_value_cont'] = [data.genes[gene_id].tissue_TiP_values[t] for gene_id in nodes]
    discr_TiP_values = [(0 if data.genes[gene_id].tissue_TiP_values[t] < 2 else data.genes[gene_id].tissue_TiP_values[t]) for gene_id in nodes]
    discr_TiP_values = [(2 if (value >= 2 and value < 3) else value) for value in discr_TiP_values]
    discr_TiP_values = [(3 if (value >= 3 and value < 9) else value) for value in discr_TiP_values]
    discr_TiP_values = [(9 if value >= 9 else value) for value in discr_TiP_values]
    g.vs['TiP_value_discrete'] = [(100 if g.vs['CG'][i] == 1 else discr_TiP_values[i]) for i in range(len(discr_TiP_values))]
    g.vs['diseases'] = ['' for i in range(len(nodes))]
    g.vs['omim_ids'] = ['' for i in range(len(nodes))]
    for CG in CGs:
        node = g.vs.select(name=CG)[0]
        for omim_id,disease_obj in data.genes[CG].diseases.items():
            if tissue in disease_obj.tissues:
                for pair in disease_obj.CG_tissue_pairs:
                    if pair[0] == CG and pair[1] == tissue:
                        node['diseases'] = node['diseases'] + '|' + disease_obj.name
                        node['diseases'] = node['diseases'].strip('|')
                        node['omim_ids'] = node['omim_ids'] + '|' + str(omim_id)
                        node['omim_ids'] = node['omim_ids'].strip('|')
                        
    TiP_PPIs = []
    for edge in edges:
        gene_a = edge[0]
        gene_b = edge[1]
        if data.genes[gene_a].tissue_TiP_values[t] >= 2 or data.genes[gene_b].tissue_TiP_values[t] >= 2:
            TiP_PPIs.append(1)
        else:
            TiP_PPIs.append(0)
    g.es['TiP_PPI'] = TiP_PPIs
                    
    g.es['v1'] = [(1 if data.genes[pair[0]].PPIs[pair[1]].version1 == 1 else 0) for pair in edges]
    g.es['v2'] = [(1 if data.genes[pair[0]].PPIs[pair[1]].version2 == 1 else 0) for pair in edges]
    g.es['v3'] = [(1 if data.genes[pair[0]].PPIs[pair[1]].version3 == 1 else 0) for pair in edges]
    g.vs['edgotyped'] = [0 for i in range(len(nodes))]
    g.es['edgotyped'] = [0 for i in range(len(edges))]
    g.es['perturbed'] = [0 for i in range(len(edges))]
    # check for each of the PPIs whether they have been successfully tested in the edgotyping experiment and 
    # whether any disease-matching and pathogenic alleles have perturbed the edge
    for tup,int_obj in data.allele_interactions.items():
        allele_id = tup[0]
        partner_id = tup[1]
        assay_id = tup[2]
        allele_obj = data.alleles[allele_id]
        pair = tuple(sorted([allele_obj.gene_id,partner_id]))
        # test whether this allele belongs to a selected causal gene
        if allele_obj.gene_id in CGs:
            print(data.genes[allele_obj.gene_id].symbol)
            # test whether this allele corresponds to any of the diseases of that causal gene for that tissue
            allele_omim_ids = set([str(omim_id) for omim_id in allele_obj.diseases.keys()])
            CG_node_obj = g.vs.select(name=allele_obj.gene_id)[0]
            cg_omim_ids = set(CG_node_obj['omim_ids'].split('|'))
#            print(allele_omim_ids,cg_omim_ids)
            if len(allele_omim_ids.intersection(cg_omim_ids)) > 0:
                print(allele_omim_ids.intersection(cg_omim_ids))
                # test whether this allele is pathogenic
                if allele_obj.allele_type and \
                   (allele_obj.allele_type.find('athogenic') > -1 or allele_obj.allele_type == 'risk factor'):
                    print(allele_obj.allele_type)
                    # test whether this interaction is part of this graph
                    if pair in edges:
                        print(pair)
                        # find the edge object in the graph for this edge
                        node_a_index = g.vs.select(name=allele_obj.gene_id)[0].index
                        node_b_index = g.vs.select(name=partner_id)[0].index
                        edge_obj = g.es.select(lambda e: (e.source==node_a_index and e.target==node_b_index) or \
                                              (e.target==node_a_index and e.source==node_b_index))[0]
                        # save the edgotyping result for this edge in the graph
                        edge_obj['edgotyped'] = 1
                        # if this edge is not perturbed by another allele already
                        if edge_obj['perturbed'] == 0:
                            # test whether this interaction was successfully tested for perturbation with this allele in 
                            # multiple assay versions and choose the best result
                            best_int_obj = None
                            for v in assay_versions:
                                if (allele_id,partner_id,v) in data.allele_interactions:
                                    new_int_obj = data.allele_interactions[allele_id,partner_id,v]
                                    if not best_int_obj:
                                        best_int_obj = new_int_obj
                                    else:
                                        if new_int_obj.GS_wt > best_int_obj.GS_wt:
                                            best_int_obj = new_int_obj
                            edge_obj['perturbed'] = best_int_obj.perturbed
                        # save the edgotyping result for this causal gene in the graph
                        CG_node_obj['edgotyped'] = 1
                        
    if len(g.vs) > 0 and len(g.es) > 0:
        tissue_graphs[tissue] = g


adipose_subcutaneous
adipose_visceral_omentum
adrenal_gland
artery_aorta
artery_coronary
artery_tibial
brain-0
DOLK
{'610768'}
Pathogenic
('ENSG00000062725', 'ENSG00000175283')
CLN6
{'204300'}
Pathogenic
('ENSG00000106089', 'ENSG00000128973')
SUCLA2
{'612073'}
Pathogenic
('ENSG00000100934', 'ENSG00000136143')
SUCLA2
{'612073'}
Pathogenic
('ENSG00000136143', 'ENSG00000170540')
SUCLA2
{'612073'}
Pathogenic
SUCLA2
{'612073'}
Pathogenic
('ENSG00000136143', 'ENSG00000177674')
SUCLA2
{'612073'}
Pathogenic
('ENSG00000136143', 'ENSG00000147676')
SUCLA2
{'612073'}
Pathogenic
('ENSG00000136143', 'ENSG00000166091')
SUCLA2
{'612073'}
Pathogenic
('ENSG00000136143', 'ENSG00000147676')
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000152422')
PNKP
{'613402'}
DOLK
{'610768'}
Pathogenic
DOLK
{'610768'}
Pathogenic
DOLK
{'610768'}
Pathogenic
('ENSG00000111262', 'ENSG00000175283')
DOLK
{'610768'}
Pathogenic
GOSR2
{'614018'}
Pathogenic
('ENSG00000103496', 'ENSG00000108433')
GOSR2
{'614018'}
Pathoge

PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000065882')
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000121864')
PNKP
{'613402'}
Pathogenic
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000145901')
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000100151')
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000172782')
PNKP
{'613402'}
Pathogenic
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000177519')
PNKP
{'613402'}
Pathogenic
('ENSG00000039650', 'ENSG00000205302')
GOSR2
{'614018'}
Pathogenic
GOSR2
{'614018'}
Pathogenic
('ENSG00000108433', 'ENSG00000197329')
GOSR2
{'614018'}
Pathogenic
('ENSG000

BAG3
{'612954', '613881'}
Conflicting interpretations of pathogenicity
('ENSG00000151929', 'ENSG00000159216')
BAG3
{'612954', '613881'}
Conflicting interpretations of pathogenicity
('ENSG00000143514', 'ENSG00000151929')
BAG3
{'612954', '613881'}
BAG3
{'612954', '613881'}
BAG3
{'612954', '613881'}
BAG3
{'612954', '613881'}
BAG3
{'612954', '613881'}
BAG3
{'612954', '613881'}
BAG3
{'612954', '613881'}
DES
DES
DES
DES
DES
DES
DES
DES
DES
DES
DES
DES
DES
DES
{'601419'}
Likely pathogenic
DES
{'601419'}
Likely pathogenic
DES
{'601419'}
Likely pathogenic
('ENSG00000131095', 'ENSG00000175084')
DES
{'601419'}
Likely pathogenic
('ENSG00000175084', 'ENSG00000175084')
DES
{'601419'}
Likely pathogenic
DES
{'601419'}
Likely pathogenic
('ENSG00000100109', 'ENSG00000175084')
DES
{'601419'}
Likely pathogenic
DES
{'601419'}
DES
{'601419'}
DES
{'601419'}
DES
{'601419'}
DES
{'601419'}
DES
{'601419'}
DES
{'601419'}
DES
{'601419'}
Conflicting interpretations of pathogenicity
DES
{'601419'}
Conflicting interp

In [5]:
for tissue,graph in tissue_graphs.items():
    outfile = outpath + tissue + '.node_attributes.txt'
    write_node_attributes(graph,outfile)
    outfile = outpath + tissue + '.edge_attributes.txt'
    write_edge_attributes(graph,outfile)

In [55]:
edgo_style = {'title': 'edgo_style2',
 'defaults': [{'visualProperty': 'COMPOUND_NODE_PADDING', 'value': 10.0},
  {'visualProperty': 'COMPOUND_NODE_SHAPE', 'value': 'ROUND_RECTANGLE'},
  {'visualProperty': 'DING_RENDERING_ENGINE_ROOT',
   'value': 'org.cytoscape.view.presentation.property.NullVisualProperty$NullDataTypeImpl@f25ab66'},
  {'visualProperty': 'EDGE',
   'value': 'DefaultVisualizableVisualProperty(id=EDGE, name=Edge Visual Property)'},
  {'visualProperty': 'EDGE_BEND', 'value': ''},
  {'visualProperty': 'EDGE_CURVED', 'value': True},
  {'visualProperty': 'EDGE_LABEL', 'value': ''},
  {'visualProperty': 'EDGE_LABEL_COLOR', 'value': '#333333'},
  {'visualProperty': 'EDGE_LABEL_FONT_FACE', 'value': 'Dialog,plain,10'},
  {'visualProperty': 'EDGE_LABEL_FONT_SIZE', 'value': 10},
  {'visualProperty': 'EDGE_LABEL_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'EDGE_LABEL_WIDTH', 'value': 200.0},
  {'visualProperty': 'EDGE_LINE_TYPE', 'value': 'SOLID'},
  {'visualProperty': 'EDGE_PAINT', 'value': '#323232'},
  {'visualProperty': 'EDGE_SELECTED', 'value': False},
  {'visualProperty': 'EDGE_SELECTED_PAINT', 'value': '#FF0000'},
  {'visualProperty': 'EDGE_SOURCE_ARROW_SELECTED_PAINT', 'value': '#FFFF00'},
  {'visualProperty': 'EDGE_SOURCE_ARROW_SHAPE', 'value': 'NONE'},
  {'visualProperty': 'EDGE_SOURCE_ARROW_SIZE', 'value': 6.0},
  {'visualProperty': 'EDGE_SOURCE_ARROW_UNSELECTED_PAINT', 'value': '#000000'},
  {'visualProperty': 'EDGE_STROKE_SELECTED_PAINT', 'value': '#FF0000'},
  {'visualProperty': 'EDGE_STROKE_UNSELECTED_PAINT', 'value': '#9B6199'},
  {'visualProperty': 'EDGE_TARGET_ARROW_SELECTED_PAINT', 'value': '#FFFF00'},
  {'visualProperty': 'EDGE_TARGET_ARROW_SHAPE', 'value': 'NONE'},
  {'visualProperty': 'EDGE_TARGET_ARROW_SIZE', 'value': 6.0},
  {'visualProperty': 'EDGE_TARGET_ARROW_UNSELECTED_PAINT', 'value': '#000000'},
  {'visualProperty': 'EDGE_TOOLTIP', 'value': ''},
  {'visualProperty': 'EDGE_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'EDGE_UNSELECTED_PAINT', 'value': '#9B6199'},
  {'visualProperty': 'EDGE_VISIBLE', 'value': True},
  {'visualProperty': 'EDGE_WIDTH', 'value': 4.0},
  {'visualProperty': 'NETWORK',
   'value': 'DefaultVisualizableVisualProperty(id=NETWORK, name=Network Visual Property)'},
  {'visualProperty': 'NETWORK_ANNOTATION_SELECTION', 'value': False},
  {'visualProperty': 'NETWORK_BACKGROUND_PAINT', 'value': '#FFFFFF'},
  {'visualProperty': 'NETWORK_CENTER_X_LOCATION', 'value': 0.0},
  {'visualProperty': 'NETWORK_CENTER_Y_LOCATION', 'value': 0.0},
  {'visualProperty': 'NETWORK_CENTER_Z_LOCATION', 'value': 0.0},
  {'visualProperty': 'NETWORK_DEPTH', 'value': 0.0},
  {'visualProperty': 'NETWORK_EDGE_SELECTION', 'value': True},
  {'visualProperty': 'NETWORK_HEIGHT', 'value': 400.0},
  {'visualProperty': 'NETWORK_NODE_SELECTION', 'value': True},
  {'visualProperty': 'NETWORK_SCALE_FACTOR', 'value': 1.0},
  {'visualProperty': 'NETWORK_SIZE', 'value': 550.0},
  {'visualProperty': 'NETWORK_TITLE', 'value': ''},
  {'visualProperty': 'NETWORK_WIDTH', 'value': 550.0},
  {'visualProperty': 'NODE',
   'value': 'DefaultVisualizableVisualProperty(id=NODE, name=Node Visual Property)'},
  {'visualProperty': 'NODE_BORDER_PAINT', 'value': '#000000'},
  {'visualProperty': 'NODE_BORDER_STROKE', 'value': 'SOLID'},
  {'visualProperty': 'NODE_BORDER_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'NODE_BORDER_WIDTH', 'value': 0.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_1',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_2',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_3',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_4',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_5',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_6',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_7',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_8',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_9',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_1',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_2',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_3',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_4',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_5',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_6',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_7',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_8',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_9',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_1', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_2', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_3', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_4', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_5', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_6', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_7', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_8', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_9', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMPAINT_1',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_1, name=Node Custom Paint 1)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_2',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_2, name=Node Custom Paint 2)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_3',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_3, name=Node Custom Paint 3)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_4',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_4, name=Node Custom Paint 4)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_5',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_5, name=Node Custom Paint 5)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_6',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_6, name=Node Custom Paint 6)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_7',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_7, name=Node Custom Paint 7)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_8',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_8, name=Node Custom Paint 8)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_9',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_9, name=Node Custom Paint 9)'},
  {'visualProperty': 'NODE_DEPTH', 'value': 0.0},
  {'visualProperty': 'NODE_FILL_COLOR', 'value': '#7FCDBB'},
  {'visualProperty': 'NODE_HEIGHT', 'value': 30.0},
  {'visualProperty': 'NODE_LABEL', 'value': ''},
  {'visualProperty': 'NODE_LABEL_COLOR', 'value': '#333333'},
  {'visualProperty': 'NODE_LABEL_FONT_FACE', 'value': 'Dialog,plain,12'},
  {'visualProperty': 'NODE_LABEL_FONT_SIZE', 'value': 14},
  {'visualProperty': 'NODE_LABEL_POSITION', 'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_LABEL_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'NODE_LABEL_WIDTH', 'value': 200.0},
  {'visualProperty': 'NODE_NESTED_NETWORK_IMAGE_VISIBLE', 'value': True},
  {'visualProperty': 'NODE_PAINT', 'value': '#787878'},
  {'visualProperty': 'NODE_SELECTED', 'value': False},
  {'visualProperty': 'NODE_SELECTED_PAINT', 'value': '#FFFF00'},
  {'visualProperty': 'NODE_SHAPE', 'value': 'ELLIPSE'},
  {'visualProperty': 'NODE_SIZE', 'value': 25.0},
  {'visualProperty': 'NODE_TOOLTIP', 'value': ''},
  {'visualProperty': 'NODE_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'NODE_VISIBLE', 'value': True},
  {'visualProperty': 'NODE_WIDTH', 'value': 70.0},
  {'visualProperty': 'NODE_X_LOCATION', 'value': 0.0},
  {'visualProperty': 'NODE_Y_LOCATION', 'value': 0.0},
  {'visualProperty': 'NODE_Z_LOCATION', 'value': 0.0}],
 'mappings': [{'mappingType': 'passthrough',
   'mappingColumn': 'symbol',
   'mappingColumnType': 'String',
   'visualProperty': 'NODE_LABEL'},
  {'mappingType': 'discrete',
   'mappingColumn': 'perturbed',
   'mappingColumnType': 'Double',
   'visualProperty': 'EDGE_LINE_TYPE',
   'map': [{'key': '0.0', 'value': 'SOLID'},
    {'key': '1.0', 'value': 'EQUAL_DASH'}]},
  {'mappingType': 'discrete',
   'mappingColumn': 'TiP_value_discrete',
   'mappingColumnType': 'Double',
   'visualProperty': 'NODE_FILL_COLOR',
   'map': [{'key': '0.0', 'value': '#BDD8F4'},
    {'key': '2.0', 'value': '#00CC99'},
    {'key': '3.0', 'value': '#00CC99'},
    {'key': '100.0', 'value': '#FF6666'},
    {'key': '9.0', 'value': '#00CC99'}]},
  {'mappingType': 'discrete',
   'mappingColumn': 'edgotyped',
   'mappingColumnType': 'Double',
   'visualProperty': 'EDGE_STROKE_UNSELECTED_PAINT',
   'map': [{'key': '0.0', 'value': '#D4C8D4'},
    {'key': '1.0', 'value': '#9B6199'}]},
  {'mappingType': 'continuous',
   'mappingColumn': 'weight',
   'mappingColumnType': 'Double',
   'visualProperty': 'NODE_SIZE',
   'points': [{'value': 0.002004008016032064,
     'lesser': '1.0',
     'equal': '10.0',
     'greater': '10.0'},
    {'value': 0.06999999705799118,
     'lesser': '30.0',
     'equal': '30.0',
     'greater': '30.0'},
    {'value': 1.0, 'lesser': '40.0', 'equal': '40.0', 'greater': '1.0'}]}]}

In [56]:
hairball_style = {'title': 'edgo_hairball',
 'defaults': [{'visualProperty': 'COMPOUND_NODE_PADDING', 'value': 10.0},
  {'visualProperty': 'COMPOUND_NODE_SHAPE', 'value': 'ROUND_RECTANGLE'},
  {'visualProperty': 'DING_RENDERING_ENGINE_ROOT',
   'value': 'org.cytoscape.view.presentation.property.NullVisualProperty$NullDataTypeImpl@f25ab66'},
  {'visualProperty': 'EDGE',
   'value': 'DefaultVisualizableVisualProperty(id=EDGE, name=Edge Visual Property)'},
  {'visualProperty': 'EDGE_BEND', 'value': ''},
  {'visualProperty': 'EDGE_CURVED', 'value': True},
  {'visualProperty': 'EDGE_LABEL', 'value': ''},
  {'visualProperty': 'EDGE_LABEL_COLOR', 'value': '#333333'},
  {'visualProperty': 'EDGE_LABEL_FONT_FACE', 'value': 'Dialog,plain,10'},
  {'visualProperty': 'EDGE_LABEL_FONT_SIZE', 'value': 10},
  {'visualProperty': 'EDGE_LABEL_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'EDGE_LABEL_WIDTH', 'value': 200.0},
  {'visualProperty': 'EDGE_LINE_TYPE', 'value': 'SOLID'},
  {'visualProperty': 'EDGE_PAINT', 'value': '#323232'},
  {'visualProperty': 'EDGE_SELECTED', 'value': False},
  {'visualProperty': 'EDGE_SELECTED_PAINT', 'value': '#FF0000'},
  {'visualProperty': 'EDGE_SOURCE_ARROW_SELECTED_PAINT', 'value': '#FFFF00'},
  {'visualProperty': 'EDGE_SOURCE_ARROW_SHAPE', 'value': 'NONE'},
  {'visualProperty': 'EDGE_SOURCE_ARROW_SIZE', 'value': 6.0},
  {'visualProperty': 'EDGE_SOURCE_ARROW_UNSELECTED_PAINT', 'value': '#000000'},
  {'visualProperty': 'EDGE_STROKE_SELECTED_PAINT', 'value': '#FF0000'},
  {'visualProperty': 'EDGE_STROKE_UNSELECTED_PAINT', 'value': '#9B6199'},
  {'visualProperty': 'EDGE_TARGET_ARROW_SELECTED_PAINT', 'value': '#FFFF00'},
  {'visualProperty': 'EDGE_TARGET_ARROW_SHAPE', 'value': 'NONE'},
  {'visualProperty': 'EDGE_TARGET_ARROW_SIZE', 'value': 6.0},
  {'visualProperty': 'EDGE_TARGET_ARROW_UNSELECTED_PAINT', 'value': '#000000'},
  {'visualProperty': 'EDGE_TOOLTIP', 'value': ''},
  {'visualProperty': 'EDGE_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'EDGE_UNSELECTED_PAINT', 'value': '#9B6199'},
  {'visualProperty': 'EDGE_VISIBLE', 'value': True},
  {'visualProperty': 'EDGE_WIDTH', 'value': 10.0},
  {'visualProperty': 'NETWORK',
   'value': 'DefaultVisualizableVisualProperty(id=NETWORK, name=Network Visual Property)'},
  {'visualProperty': 'NETWORK_ANNOTATION_SELECTION', 'value': False},
  {'visualProperty': 'NETWORK_BACKGROUND_PAINT', 'value': '#FFFFFF'},
  {'visualProperty': 'NETWORK_CENTER_X_LOCATION', 'value': 0.0},
  {'visualProperty': 'NETWORK_CENTER_Y_LOCATION', 'value': 0.0},
  {'visualProperty': 'NETWORK_CENTER_Z_LOCATION', 'value': 0.0},
  {'visualProperty': 'NETWORK_DEPTH', 'value': 0.0},
  {'visualProperty': 'NETWORK_EDGE_SELECTION', 'value': True},
  {'visualProperty': 'NETWORK_HEIGHT', 'value': 400.0},
  {'visualProperty': 'NETWORK_NODE_SELECTION', 'value': True},
  {'visualProperty': 'NETWORK_SCALE_FACTOR', 'value': 1.0},
  {'visualProperty': 'NETWORK_SIZE', 'value': 550.0},
  {'visualProperty': 'NETWORK_TITLE', 'value': ''},
  {'visualProperty': 'NETWORK_WIDTH', 'value': 550.0},
  {'visualProperty': 'NODE',
   'value': 'DefaultVisualizableVisualProperty(id=NODE, name=Node Visual Property)'},
  {'visualProperty': 'NODE_BORDER_PAINT', 'value': '#000000'},
  {'visualProperty': 'NODE_BORDER_STROKE', 'value': 'SOLID'},
  {'visualProperty': 'NODE_BORDER_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'NODE_BORDER_WIDTH', 'value': 0.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_1',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_2',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_3',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_4',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_5',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_6',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_7',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_8',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_9',
   'value': 'org.cytoscape.ding.customgraphics.NullCustomGraphics,0,[ Remove Graphics ],'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_1',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_2',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_3',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_4',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_5',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_6',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_7',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_8',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_POSITION_9',
   'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_1', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_2', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_3', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_4', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_5', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_6', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_7', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_8', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMGRAPHICS_SIZE_9', 'value': 50.0},
  {'visualProperty': 'NODE_CUSTOMPAINT_1',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_1, name=Node Custom Paint 1)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_2',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_2, name=Node Custom Paint 2)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_3',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_3, name=Node Custom Paint 3)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_4',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_4, name=Node Custom Paint 4)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_5',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_5, name=Node Custom Paint 5)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_6',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_6, name=Node Custom Paint 6)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_7',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_7, name=Node Custom Paint 7)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_8',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_8, name=Node Custom Paint 8)'},
  {'visualProperty': 'NODE_CUSTOMPAINT_9',
   'value': 'DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_9, name=Node Custom Paint 9)'},
  {'visualProperty': 'NODE_DEPTH', 'value': 0.0},
  {'visualProperty': 'NODE_FILL_COLOR', 'value': '#7FCDBB'},
  {'visualProperty': 'NODE_HEIGHT', 'value': 30.0},
  {'visualProperty': 'NODE_LABEL', 'value': ''},
  {'visualProperty': 'NODE_LABEL_COLOR', 'value': '#333333'},
  {'visualProperty': 'NODE_LABEL_FONT_FACE', 'value': 'Dialog,plain,12'},
  {'visualProperty': 'NODE_LABEL_FONT_SIZE', 'value': 14},
  {'visualProperty': 'NODE_LABEL_POSITION', 'value': 'C,C,c,0.00,0.00'},
  {'visualProperty': 'NODE_LABEL_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'NODE_LABEL_WIDTH', 'value': 200.0},
  {'visualProperty': 'NODE_NESTED_NETWORK_IMAGE_VISIBLE', 'value': True},
  {'visualProperty': 'NODE_PAINT', 'value': '#787878'},
  {'visualProperty': 'NODE_SELECTED', 'value': False},
  {'visualProperty': 'NODE_SELECTED_PAINT', 'value': '#FFFF00'},
  {'visualProperty': 'NODE_SHAPE', 'value': 'ELLIPSE'},
  {'visualProperty': 'NODE_SIZE', 'value': 25.0},
  {'visualProperty': 'NODE_TOOLTIP', 'value': ''},
  {'visualProperty': 'NODE_TRANSPARENCY', 'value': 255},
  {'visualProperty': 'NODE_VISIBLE', 'value': True},
  {'visualProperty': 'NODE_WIDTH', 'value': 70.0},
  {'visualProperty': 'NODE_X_LOCATION', 'value': 0.0},
  {'visualProperty': 'NODE_Y_LOCATION', 'value': 0.0},
  {'visualProperty': 'NODE_Z_LOCATION', 'value': 0.0}],
 'mappings': [{'mappingType': 'discrete',
   'mappingColumn': 'edgotyped',
   'mappingColumnType': 'Double',
   'visualProperty': 'EDGE_WIDTH',
   'map': [{'key': '0.0', 'value': '10.0'}, {'key': '1.0', 'value': '15.0'}]},
  {'mappingType': 'discrete',
   'mappingColumn': 'perturbed',
   'mappingColumnType': 'Double',
   'visualProperty': 'EDGE_LINE_TYPE',
   'map': [{'key': '0.0', 'value': 'SOLID'},
    {'key': '1.0', 'value': 'EQUAL_DASH'}]},
  {'mappingType': 'discrete',
   'mappingColumn': 'TiP_value_discrete',
   'mappingColumnType': 'Double',
   'visualProperty': 'NODE_FILL_COLOR',
   'map': [{'key': '0.0', 'value': '#BFDAF4'},
    {'key': '2.0', 'value': '#00CC99'},
    {'key': '3.0', 'value': '#00CC99'},
    {'key': '100.0', 'value': '#FF6666'},
    {'key': '9.0', 'value': '#00CC99'}]},
  {'mappingType': 'discrete',
   'mappingColumn': 'TiP_value_discrete',
   'mappingColumnType': 'Double',
   'visualProperty': 'NODE_SIZE',
   'map': [{'key': '0.0', 'value': '15.0'},
    {'key': '2.0', 'value': '30.0'},
    {'key': '9.0', 'value': '30.0'},
    {'key': '3.0', 'value': '30.0'},
    {'key': '100.0', 'value': '30.0'}]},
  {'mappingType': 'discrete',
   'mappingColumn': 'edgotyped',
   'mappingColumnType': 'Double',
   'visualProperty': 'EDGE_STROKE_UNSELECTED_PAINT',
   'map': [{'key': '0.0', 'value': '#D4C8D4'},
    {'key': '1.0', 'value': '#9B6199'}]}]}

In [14]:
cy.session.delete()
edgo_style_obj = cy.style.create(name='edgo_style2',original_style=edgo_style)
hairball_style_obj = cy.style.create(name='edgo_hairball',original_style=hairball_style)

network_id_map = {}

for tissue,g in tissue_graphs.items():
    network = cy.network.create_from_igraph(g,name=tissue)
    cy.layout.apply(name='force-directed', network=network)
    cy.style.apply(hairball_style_obj,network=network)
    pdf = network.get_pdf()
    target = open(outpath + tissue + '.network.hairball.pdf','wb')
    target.write(pdf)
    target.close()

    cy.style.apply(edgo_style_obj,network=network)
    pdf = network.get_pdf()
    target = open(outpath + tissue + '.network.zoom.pdf','wb')
    target.write(pdf)
    target.close()
    
    network_id_map[tissue] = network.get_id()
